# Sample Code

## 基礎建設

In [1]:
import pandas as pd
import gzip, json

def parse(path):
    g = gzip.open(path, 'rb')
    for l in g:
        yield json.loads(l)

def getDF(path):
    i = 0
    df = {}
    for d in parse(path):
        df[i] = d
        i += 1
    return pd.DataFrame.from_dict(df, orient='index')

## 載入資料

In [2]:
!wget http://deepyeti.ucsd.edu/jianmo/amazon/categoryFilesSmall/All_Beauty.csv
!wget http://deepyeti.ucsd.edu/jianmo/amazon/metaFiles2/meta_All_Beauty.json.gz

--2022-01-09 02:57:40--  http://deepyeti.ucsd.edu/jianmo/amazon/categoryFilesSmall/All_Beauty.csv
Resolving deepyeti.ucsd.edu (deepyeti.ucsd.edu)... 169.228.63.50
Connecting to deepyeti.ucsd.edu (deepyeti.ucsd.edu)|169.228.63.50|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 15499476 (15M) [application/octet-stream]
Saving to: ‘All_Beauty.csv’

All_Beauty.csv      100%[===================>]  14.78M  9.47MB/s    in 1.6s    

2022-01-09 02:57:42 (9.47 MB/s) - ‘All_Beauty.csv’ saved [15499476/15499476]

--2022-01-09 02:57:42--  http://deepyeti.ucsd.edu/jianmo/amazon/metaFiles2/meta_All_Beauty.json.gz
Resolving deepyeti.ucsd.edu (deepyeti.ucsd.edu)... 169.228.63.50
Connecting to deepyeti.ucsd.edu (deepyeti.ucsd.edu)|169.228.63.50|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10329961 (9.9M) [application/octet-stream]
Saving to: ‘meta_All_Beauty.json.gz’

meta_All_Beauty.jso 100%[===================>]   9.85M  7.54MB/s    in 1.3s    

202

In [3]:
metadata = getDF('/content/meta_All_Beauty.json.gz')
ratings = pd.read_csv('/content/All_Beauty.csv', names=['asin', 'reviewerID', 'overall', 'unixReviewTime'], header=None)

In [4]:
metadata.head()

,category,tech1,description,fit,title,also_buy,tech2,brand,feature,rank,also_view,details,main_cat,similar_item,date,price,asin,imageURL,imageURLHighRes
0,[],,[Loud 'N Clear Personal Sound Amplifier allows...,,Loud 'N Clear&trade; Personal Sound Amplifier,[],,idea village,[],"2,938,573 in Beauty & Personal Care (",[],{'ASIN: ': '6546546450'},All Beauty,,,,6546546450,[],[]
1,[],,[No7 Lift & Luminate Triple Action Serum 50ml ...,,No7 Lift &amp; Luminate Triple Action Serum 50...,"[B01E7LCSL6, B008X5RVME]",,,[],"872,854 in Beauty & Personal Care (",[],"{'Shipping Weight:': '0.3 ounces (', 'ASIN: ':...",All Beauty,"class=""a-bordered a-horizontal-stripes a-spa...",,$44.99,7178680776,[],[]
2,[],,[No7 Stay Perfect Foundation now stays perfect...,,No7 Stay Perfect Foundation Cool Vanilla by No7,[],,No7,[],"956,696 in Beauty & Personal Care (","[B01B8BR0O8, B01B8BR0NO, B014MHXXM8]","{'Shipping Weight:': '3.5 ounces (', 'ASIN: ':...",All Beauty,,,$28.76,7250468162,[],[]
3,[],,[],,Wella Koleston Perfect Hair Colour 44/44 Mediu...,[B0041PBXX8],,,[],"1,870,258 in Beauty & Personal Care (",[],"{' Item Weight: ': '1.76 ounces', 'Sh...",All Beauty,,,,7367905066,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...
4,[],,[Lacto Calamine Skin Balance Daily Nourishing ...,,Lacto Calamine Skin Balance Oil control 120 ml...,[],,Pirmal Healthcare,[],"67,701 in Beauty & Personal Care (","[3254895630, B007VL1D9S, B00EH9A0RI, B0773MBG4...","{'Shipping Weight:': '12 ounces (', 'ASIN: ': ...",All Beauty,,,$12.15,7414204790,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...


In [5]:
ratings.head()

,asin,reviewerID,overall,unixReviewTime
0,0143026860,A1V6B6TNIC10QE,1.0,1424304000
1,0143026860,A2F5GHSXFQ0W6J,4.0,1418860800
2,0143026860,A1572GUYS7DGSR,4.0,1407628800
3,0143026860,A1PSGLFK1NSVO,5.0,1362960000
4,0143026860,A6IKXKZMTKGSC,5.0,1324771200


## 資料整理

In [6]:
ratings['DATE'] = pd.to_datetime(ratings['unixReviewTime'], unit='s')

## 資料切分

In [7]:
ratings_trainings = ratings[
    (ratings['DATE'] < '2018-09-01')
]
ratings_testings = ratings[
    (ratings['DATE'] >= '2018-09-01') & 
    (ratings['DATE'] <= '2018-09-30')
]
ratings_testings_by_user = ratings_testings.groupby('reviewerID').agg(list).reset_index()[['reviewerID', 'asin']].to_dict('records')
ratings_testings_by_user = { rating['reviewerID']: rating['asin'] for rating in ratings_testings_by_user }
users = list(ratings_testings_by_user.keys())

## 產生推薦

In [8]:
!pip install lightfm

     |████████████████████████████████| 310 kB 4.1 MB/s 
  Created wheel for lightfm: filename=lightfm-1.16-cp37-cp37m-linux_x86_64.whl size=705361 sha256=bd77c8a6f3b3c7748e1f584619b4abd43f7b59f84d6ccd0f64c4f0495662e853
  Stored in directory: /root/.cache/pip/wheels/f8/56/28/5772a3bd3413d65f03aa452190b00898b680b10028a1021914
Successfully built lightfm


In [9]:
from lightfm import LightFM
import numpy as np

def recommend_items(model, data, user_id, item_list, k):
    
    # 取得item數量
    n_users, n_items = data.shape

    # 預測user對所有item的評分並進行推薦
    scores = model.predict(user_id, np.arange(n_items))
    top_items = item_list[np.argsort(-scores)]
    return top_items[:k].tolist()

In [25]:
from lightfm.data import Dataset
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

pd.options.mode.chained_assignment = None
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')


def recommender(training_data, users=[], k=10, rule_based = False, add_test_user = False, add_item_feature = False, metadata = ''):
    '''
    * training_data: dataframe 輸入的訓練資料集（2018-09-01 以前資料）
    * users: [] 需要被推薦的使用者
    * k: int 每個使用者需要推薦的商品數
    * recommendations: dict
      {
          使用者一： [推薦商品一, 推薦商品二, ...],
          使用者二： [...], ...
      }
    '''
    # 移除使用者對商品的重複評論，只留最新
    training_data = training_data.sort_values("DATE", ascending=False).groupby(['reviewerID', 'asin']).head(1)
    recommendations = {}
    '''
    mf-based, lightFM
    '''
    # Rule based
    if rule_based and isinstance(training_data, pd.DataFrame):
        hot = training_data[(training_data['DATE'] >= '2018-08-01') & (training_data['DATE'] < '2018-09-01') & (training_data['overall'] >= 4)]
        hot = pd.DataFrame(hot.asin.value_counts())
        hot.reset_index(inplace=True)
        hot.columns = ['asin', 'count']
    
    # 取出評分矩陣所需的訓練資料並以lightfm的Dataset形式存取
    training_data = training_data[["reviewerID","asin","overall"]]
    dataset = Dataset()

    if add_test_user:
        dataset.fit(np.append(training_data.reviewerID.unique(), users, 0), training_data.asin.unique())
    else:
        dataset.fit(training_data.reviewerID.unique(), training_data.asin.unique())
    
    interactions_matrix, weights_matrix = dataset.build_interactions([tuple(i) for i in training_data.values])


    # 取得user與item的mapping資料
    user_id_map, user_feature_map, item_id_map, item_feature_map = dataset.mapping()
    item_list = np.array(list(item_id_map.keys())).astype(object)

    # 訓練模型
    if add_item_feature:
        assert isinstance(metadata, pd.DataFrame), "Please input the item's data"
        # 篩選資料
        metadata = metadata[metadata.asin.isin(training_data.asin.tolist())]
        metadata.reset_index(drop=True, inplace=True)
        
        # set lemmatizer
        lemmatizer = WordNetLemmatizer()
        
        # 處理掉description 的中括號
        metadata['description'] = metadata['description'].apply(lambda x: ' '.join(x))

        # 處理rank
        rankdata = metadata['rank'].str.split(' ', n=1, expand=True)
        metadata['rank_no'], metadata['rank'] = rankdata.iloc[:,0], rankdata.iloc[:,1]

        # 合併文字欄位
        metadata['all_text'] = metadata['description'] + ' ' + metadata['title'] # + ' ' + metadata['rank'] + ' ' + metadata['brand']

        # 去除特殊符號，除了數字之間的
        import re
        regex = r"(?<!\d)[\W](?!\d)"
        metadata['all_text'] = metadata['all_text'].apply(lambda x: re.sub(regex, ' ', str(x)))
        metadata['rank'] = metadata['rank'].apply(lambda x: re.sub(regex, ' ', str(x)))
        metadata['brand'] = metadata['brand'].apply(lambda x: re.sub(regex, ' ', str(x)))

        # 轉小寫
        metadata['all_text'] = metadata['all_text'].str.lower()
        metadata['rank'] = metadata['rank'].str.lower() 
        metadata['brand'] = metadata['brand'].str.lower()         

        # remove stopword
        def remove_stopword(sentence):
            tokens = nltk.word_tokenize(sentence)
            result = [lemmatizer.lemmatize(voca) for voca in tokens if not voca in stopwords.words('english')]
            return result

        metadata['all_text'] = metadata['all_text'].apply(lambda x: remove_stopword(x))

        # merge column
        for i in range(len(metadata)):
            try:
                metadata['all_text'][i] = metadata['all_text'][i].append(metadata['rank'][i]).append(metadata['brand'][i])
            except:
                metadata['all_text'][i] = []

        # 建立隱性標籤
        item_features = dataset.build_item_features([tuple(x) for x in metadata[['asin','all_text']].values])

        # 訓練混合模型（Hybrid Model）加入 user / item feature
        model = LightFM(loss='warp', item_alpha=1e-6, no_components=100)
        model = model.fit(interactions_matrix, sample_weight = weights_matrix, item_features=item_features, epochs=20, num_threads=8)

    else:
        model = LightFM(loss='warp', item_alpha=1e-6, no_components=100)
        model = model.fit(interactions_matrix, sample_weight = weights_matrix, epochs=20, num_threads = 8)

    # 推薦
    for user in users:
        try:
            recommendations[user] = recommend_items(model, interactions_matrix, user_id_map[user], item_list, k)
        except:
            if rule_based:
                recommendations[user] = hot.asin[:k].tolist()
            else:
                recommendations[user] = []

    return recommendations

# 推薦成果：可看出冷啟動問題嚴重，針對未曾出現的使用者無法有鑑別的個別推薦，實務上可能會主動請使用者先評分再訓練進行推薦
#          除此之外也能嘗試使用混合模型（Hybrid Model）加入user / item feature進行訓練

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


## 結果評估

In [26]:
def evaluate(ratings_testings_by_user={}, ratings_by_user={}, method=None):
    '''
    * ratings_testings_by_user: dict 真實被購買的商品資料（2018-09-01 以後資料）
    * ratings_by_user: dict 利用訓練資料學習的推薦商品
    * method: str
    * score: float
    '''
    total = 0
    for d in ratings_testings_by_user:
        if d in ratings_by_user:
            total += len(set(ratings_by_user[d]) & set(ratings_testings_by_user[d]))

    score = total / len(ratings_testings)
    return score



In [27]:
from itertools import product
ks = [10, 20, 30]
rule_baseds = [False, True]
add_test_users = [False, True]
add_item_features = [False, True]
for k, rule_based, add_test_user, add_item_feature in product(ks, rule_baseds, add_test_users, add_item_features): 
    ratings_by_user = recommender(ratings_trainings, 
                                  users, 
                                  k=k, 
                                  rule_based = rule_based, 
                                  add_test_user = add_test_user, 
                                  add_item_feature = add_item_feature, 
                                  metadata = metadata)
    score = round(evaluate(ratings_testings_by_user, ratings_by_user),3)
    print(f"推薦數 {k}, 混合 Rule-based: {str(rule_based):<5s}, 把 test user 放進模型訓練: {str(add_test_user):<5s}, 加入商品特徵: {str(add_item_feature):<5s}, recall: {score}")

推薦數 10, 混合 Rule-based: False, 把 test user 放進模型訓練: False, 加入商品特徵: False, recall: 0.0
推薦數 10, 混合 Rule-based: False, 把 test user 放進模型訓練: False, 加入商品特徵: True , recall: 0.002
推薦數 10, 混合 Rule-based: False, 把 test user 放進模型訓練: True , 加入商品特徵: False, recall: 0.085
推薦數 10, 混合 Rule-based: False, 把 test user 放進模型訓練: True , 加入商品特徵: True , recall: 0.081
推薦數 10, 混合 Rule-based: True , 把 test user 放進模型訓練: False, 加入商品特徵: False, recall: 0.137
推薦數 10, 混合 Rule-based: True , 把 test user 放進模型訓練: False, 加入商品特徵: True , recall: 0.139
推薦數 10, 混合 Rule-based: True , 把 test user 放進模型訓練: True , 加入商品特徵: False, recall: 0.083
推薦數 10, 混合 Rule-based: True , 把 test user 放進模型訓練: True , 加入商品特徵: True , recall: 0.083
推薦數 20, 混合 Rule-based: False, 把 test user 放進模型訓練: False, 加入商品特徵: False, recall: 0.002
推薦數 20, 混合 Rule-based: False, 把 test user 放進模型訓練: False, 加入商品特徵: True , recall: 0.002
推薦數 20, 混合 Rule-based: False, 把 test user 放進模型訓練: True , 加入商品特徵: False, recall: 0.085
推薦數 20, 混合 Rule-based: False, 把 test user 放進模型訓練: True ,